In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load c14
c14="DDW-0000C-14.xls"
df_c14=pd.read_excel(c14,skiprows=[0,2,3,4,5,6])

#preprocess c14
df_c14 = df_c14[["State","Area Name","Age-group","Total "]]
df_c14.rename(columns={"State":"State Code","Area Name":"Name","Total ":"TOT_P"},inplace=True)
df_c14 =df_c14[(df_c14["Age-group"]!="0-4") & (df_c14["Age-group"]!="All ages")]

In [3]:
#form 30-49 age group by adding the respective smaller groups
df_c14_30_49=df_c14.loc[df_c14['Age-group'].isin(['30-34','35-39','40-44','45-49'])].groupby(["State Code","Name"],as_index=False)["TOT_P"].sum()
df_c14_30_49["Age-group"]="30-49"

In [4]:
#form 50-69 age group by adding the respective smaller groups
df_c14_50_69=df_c14.loc[df_c14['Age-group'].isin(['50-54','55-59','60-64','65-69'])].groupby(["State Code","Name"],as_index=False)["TOT_P"].sum()
df_c14_50_69["Age-group"]="50-69"

In [5]:
#form 70+ age group by adding the respective smaller groups
df_c14_70Plus =df_c14.loc[df_c14['Age-group'].isin(['70-74','75-79','80+'])].groupby(["State Code","Name"],as_index=False)["TOT_P"].sum()
df_c14_70Plus["Age-group"]="70+"

In [6]:
df_sum_merge = pd.concat([df_c14_30_49,df_c14_50_69,df_c14_70Plus])
df_sum_merge.sort_values(by=["State Code","Age-group"],ignore_index=True,inplace=True)

In [7]:
df_c14=df_c14.loc[df_c14['Age-group'].isin(['5-9','10-14','15-19','20-24','25-29','Age not stated'])]

In [8]:
#concat all the age groups dataframes for single df
df_c14=pd.concat([df_c14,df_sum_merge])

In [9]:
df_c14.sort_values(by=["State Code","Age-group"],inplace=True,ignore_index=True)
del df_c14['Name']
df_c14

,State Code,Age-group,TOT_P
0,0,10-14,132709212
1,0,15-19,120526449
2,0,20-24,111424222
3,0,25-29,101413965
4,0,30-49,308492074
...,...,...,...
319,35,30-49,119161
320,35,5-9,30454
321,35,50-69,47692
322,35,70+,8822


In [10]:
#load c18
c18="DDW-C18-0000.xlsx"
df_c18=pd.read_excel(c18,skiprows=[0,4,5])

#preprocess c18
df_c18=df_c18[(df_c18["Total/"]=="Total")&(df_c18["Age-group"]!="Total")]
df_c18=df_c18[["State","Area Name","Age-group","Number speaking third language"]]
df_c18.rename(columns = {'Area Name':'Name','State':'State Code'},inplace=True)
df_c18

,State Code,Name,Age-group,Number speaking third language
3,00,INDIA,5-9,1844108
4,00,INDIA,10-14,7254335
5,00,INDIA,15-19,12626717
6,00,INDIA,20-24,12834334
7,00,INDIA,25-29,10857171
...,...,...,...,...
1057,35,ANDAMAN & NICOBAR ISLANDS,25-29,9716
1058,35,ANDAMAN & NICOBAR ISLANDS,30-49,27080
1059,35,ANDAMAN & NICOBAR ISLANDS,50-69,7803
1060,35,ANDAMAN & NICOBAR ISLANDS,70+,771


In [11]:
df_c14["State Code"]=df_c14["State Code"].astype(int)
df_c18["State Code"]=df_c18["State Code"].astype(int)

In [12]:
#merge c18 and c14
df_merge = pd.merge(df_c18,df_c14,how='inner',left_on=['State Code','Age-group'],right_on = ['State Code','Age-group'])
df_merge

,State Code,Name,Age-group,Number speaking third language,TOT_P
0,0,INDIA,5-9,1844108,126928126
1,0,INDIA,10-14,7254335,132709212
2,0,INDIA,15-19,12626717,120526449
3,0,INDIA,20-24,12834334,111424222
4,0,INDIA,25-29,10857171,101413965
...,...,...,...,...,...
319,35,ANDAMAN & NICOBAR ISLANDS,25-29,9716,41722
320,35,ANDAMAN & NICOBAR ISLANDS,30-49,27080,119161
321,35,ANDAMAN & NICOBAR ISLANDS,50-69,7803,47692
322,35,ANDAMAN & NICOBAR ISLANDS,70+,771,8822


In [13]:
#calculating percentage of three language speakers 
df_merge["percentage"]=((df_merge["Number speaking third language"] *100)/df_merge["TOT_P"]).astype(float)

#sorting percentages with respect to each state code
df_merge.sort_values(by=["State Code","percentage"],inplace=True)
df_merge.head(25)

,State Code,Name,Age-group,Number speaking third language,TOT_P,percentage
0,0,INDIA,5-9,1844108,126928126,1.452876
7,0,INDIA,70+,1954539,39730350,4.919511
1,0,INDIA,10-14,7254335,132709212,5.466339
8,0,INDIA,Age not stated,255131,4489802,5.682455
6,0,INDIA,50-69,10916361,152333999,7.166070
5,0,INDIA,30-49,27466884,308492074,8.903595
2,0,INDIA,15-19,12626717,120526449,10.476304
4,0,INDIA,25-29,10857171,101413965,10.705795
3,0,INDIA,20-24,12834334,111424222,11.518442
9,1,JAMMU & KASHMIR,5-9,73589,1411973,5.211785


In [14]:
#taking highest percentage group in each state

df_final=df_merge[8: :9]
df_final.rename(columns={'State Code':"state/ut","Age-group":"age-group"},inplace=True)
df_final = df_final[["state/ut","age-group","percentage"]]
df_final

,state/ut,age-group,percentage
3,0,20-24,11.518442
12,1,20-24,29.580167
21,2,20-24,8.206521
29,3,15-19,45.763949
42,4,50-69,42.774903
48,5,20-24,3.094835
56,6,15-19,7.318620
70,7,70+,14.999231
74,8,15-19,2.884065
84,9,20-24,2.443396


In [15]:
#load the final data frame to csv
outname = 'age-india.csv'
outfiles = './Output_Files'
if not os.path.exists(outfiles):
    os.mkdir(outfiles)
outdir=outfiles+'/Q5'
if not os.path.exists(outdir):
    os.mkdir(outdir)
filepath = os.path.join(outdir, outname)    

df_final.to_csv(filepath,index=False)